In [ ]:
!pip install transformers -q
import pandas as pd
import numpy as np
import torch
from transformers import pipeline
# from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
from tqdm import tqdm

In [3]:
# model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# sentiment_analysis = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True, device=0)


In [39]:
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english", return_all_scores=True, device=0)
print(sentiment_analysis(["I love this!", "I hate this."]))

[[{'label': 'NEGATIVE', 'score': 0.0011343901278451085}, {'label': 'POSITIVE', 'score': 0.9988656640052795}], [{'label': 'NEGATIVE', 'score': 0.999479353427887}, {'label': 'POSITIVE', 'score': 0.0005206770729273558}]]


In [ ]:
def perform_sentiment_analysis(docs):
    results = []
    for pred in sentiment_analysis(docs):
      score = pred['score']
      if score < 0.6 and score > 0.4:
        results.append('NEUTRAL')
      else:
        results.append(pred['label'])
    return results


In [ ]:
docs = ["They forgot to mention the outside gates were shut between certain times.",
"Not a problem in itself as it's the same key as the main doors.",
"However, as we had to check out at 8am to get to the airport we were told to leave the keys in the flat and shut the door behind us.",
"Luckily I checked to see if the gates were open (they needed the key to open from the inside as well).",
"The gates were still shut, so we had to open the gates, one of us stayed there to keep open and make sure no one came in while the other dropped the keys back in the flat.Otherwise everything else was great.",
"Great location.",
"Very clean apartment and week equipped.",
"Comfortable beds, spacious  rooms.",
"Friendly staff.",
"All good All",
"Just one little negative - the coffee pods provided were the wrong size.",
"However, one short text to the owner, and everything was sorted quickly.Thank you!",
"I appreciated the spaciousness of the layout as well as the quality of the equipment provided.",
"It was also ideally situated for my trip.",
"The staff is very nice and helpful  All it's modern, restrooms, bedroom, rooftop.",
"The elevator broke down on the last nite and wasn’t up till the next morning.",
"Not a great variety of food around and it’s a bit of walking although hosts gives u a lot of location and data of restaurants.",
"I would advise families with toddlers to stay away.",
"It’s an apartment in a 1970s flat.",
"A good mix of old and new.",
"The apartment itself was ‘state of the art’ almost perfect interiors and design.",
"Hosts were very helpful for an unfortunate event for me.",
"Not really difficult to find, but I feel directions should be a bit more foolproof."
"Would have been perfect except I had to struggle the last nite and my check out morning due to faulty elevator.",
"Had luggages and a baby to settle.",
"No criticism.",
"Very helpful & friendly staff at reception.",
"Fast check-in and check-out.",
"Very close to the main train and metro station with many stores.",
"Clean and comfortable.",
"Breakfast is not so much choices.",
"Near to Airport.",
"The shower was not working very well."]

In [34]:
#evaluate the output
sentiment_analysis(["This hotel is not that good.", "Yo mama"])

[{'label': 'NEGATIVE', 'score': 0.9994877576828003},
 {'label': 'POSITIVE', 'score': 0.9792571663856506}]

In [35]:
score_dict = {
    'POSITIVE': 1, 'NEGATIVE':-1
}

In [40]:
sentence_df = pd.read_csv('stratified_random_samples.csv')
sentence_df.index = sentence_df['Unnamed: 0']
sentence_df = sentence_df.rename({'0':"sentence"}, axis = 1)
sentence_df = sentence_df.drop('Unnamed: 0', axis=1)

In [41]:
sentence_df.head(10)

,sentence
Unnamed: 0,
6ca8df09-b670-4c97-9bd9-c3fb55f20ec4,Nothing :-)
be59260a-b214-4955-8909-3038ee184a68,Nothing comes to mind
4818571c-5e97-46d1-b78c-3f67c9407d98,Spend a little more.
b511befb-9f5f-48fd-a9e0-b472d4eb845a,Nothing really.
447c6e4d-6a67-42d6-8e2c-dd1a2b47526c,Nothing to note
38ae70da-67dd-452e-b8f6-e0162f475aa5,Otherwise nothing else.
208c8103-ee30-46fc-aebe-d452b57f54ed,It still didn’t work.
196c403f-f5c6-46df-a872-297b7363c8d1,A bit noisy from neighbours.
57e68959-7aeb-43a2-b28d-7785f4b9e705,Nothing in their control but a very loud train...


In [42]:
batch_size=64
remaining = len(sentence_df)
i = 0
scores = []
while remaining > 0:
  if remaining > batch_size:
    batch = list(sentence_df.iloc[i*batch_size:(i+1)*batch_size].sentence.values)
    scores.extend(sentiment_analysis(batch))
  else:
    batch = list(sentence_df.iloc[i*batch_size:].sentence.values)
    scores.extend(sentiment_analysis(batch))
  remaining -= batch_size
  i+= 1

In [48]:
sentence_df['sentiment_score'] = scores
sentence_df.sentiment_score.head(10)

Unnamed: 0
6ca8df09-b670-4c97-9bd9-c3fb55f20ec4    [{'label': 'NEGATIVE', 'score': 0.998745262622...
be59260a-b214-4955-8909-3038ee184a68    [{'label': 'NEGATIVE', 'score': 0.996769845485...
4818571c-5e97-46d1-b78c-3f67c9407d98    [{'label': 'NEGATIVE', 'score': 0.005380967631...
b511befb-9f5f-48fd-a9e0-b472d4eb845a    [{'label': 'NEGATIVE', 'score': 0.996590256690...
447c6e4d-6a67-42d6-8e2c-dd1a2b47526c    [{'label': 'NEGATIVE', 'score': 0.999395012855...
38ae70da-67dd-452e-b8f6-e0162f475aa5    [{'label': 'NEGATIVE', 'score': 0.999388873577...
208c8103-ee30-46fc-aebe-d452b57f54ed    [{'label': 'NEGATIVE', 'score': 0.999497890472...
196c403f-f5c6-46df-a872-297b7363c8d1    [{'label': 'NEGATIVE', 'score': 0.011973983608...
57e68959-7aeb-43a2-b28d-7785f4b9e705    [{'label': 'NEGATIVE', 'score': 0.998017311096...
1a486449-50e6-4640-8e59-8dc5bdf5c109    [{'label': 'NEGATIVE', 'score': 0.993081510066...
Name: sentiment_score, dtype: object

In [49]:
def calculate_final_score(scores):
  final_score = 0
  for score in scores:
    final_score += (score_dict[score['label']]*score['score'])
  return final_score
calculate_final_score(sentiment_analysis("No view from my room.")[0])

-0.9988092263811268

In [50]:
sentence_df['sentiment_score'] = sentence_df.sentiment_score.apply(lambda x: calculate_final_score(x)) 


In [52]:
sentence_df.sort_values(by='sentiment_score', ascending=False, inplace=True)

In [53]:
sentence_df

,sentence,sentiment_score
Unnamed: 0,,
5018fe0c-1a7e-4d87-928d-b3e46e55502e,For the price we paid we did not have high exp...,0.997879
460943b6-6b51-4017-9603-d552df999013,I can’t fault anything about the accommodation...,0.997875
a6eeaaa7-249a-4bbf-8b09-633668dd7070,Breakfast was delicious - quality local produc...,0.997874
ddaffbb7-262b-40e5-96d3-91576d3b7462,"We loved this place, 5+Great helpful friendly ...",0.997873
85f77a24-9f30-4971-8bce-78b9bf49f1b7,Exceptional value for money during peak season...,0.997870
...,...,...
05b95684-4f04-45dc-acdf-861355ee72a2,Unacceptable level of cleanliness on the whole.,-0.999021
04d14230-e8be-4b80-bf4e-12578d45d7b6,"Shower dirty and leaked, staff unfriendly and ...",-0.999023
0a3b82ef-55df-4347-ac0d-9cd1ecc85788,We walked in from the street to get our room k...,-0.999026


In [55]:
sentence_df.to_csv('stratified_random_samples_sorted.csv', index=True)

TypeError: ignored